## NNアプローチ (Plan B): MLPモデルの実装・評価 (特徴量セットB)

Jane Streetコンペの知見に基づき、MLPモデルを試します。
特徴量セットBのCSVファイルを読み込み、データを準備して実行します。
TensorFlow (Keras) が必要です: pip install tensorflow


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

# --- 1. データの読み込み ---
print("--- 1. CSVデータの読み込み ---")
csv_filename = 'BTCUSDT_5m_processed_set_b.csv' # 特徴量セットBを保存したファイル名
try:
    df_processed_b_loaded = pd.read_csv(csv_filename, index_col='timestamp', parse_dates=True)
    print(f"'{csv_filename}' を読み込みました。")
    df_processed_b_loaded.info(verbose=False, memory_usage='deep')
except FileNotFoundError:
    print(f"エラー: ファイル '{csv_filename}' が見つかりません。")
    df_processed_b_loaded = None
except Exception as e:
    print(f"CSV読み込み中にエラーが発生しました: {e}")
    df_processed_b_loaded = None

# --- 2. データ分割 ---
if df_processed_b_loaded is not None and not df_processed_b_loaded.empty:
    print("\n--- 2. データ分割 ---")
    # 特徴量とターゲットの指定 (元のOHLCV等は除外)
    exclude_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'wclprice', 'target']
    features_b = [col for col in df_processed_b_loaded.columns if col not in exclude_cols]
    print(f"使用する特徴量の数: {len(features_b)}")

    X_b = df_processed_b_loaded[features_b]
    y_b = df_processed_b_loaded['target']

    # 分割 (時系列考慮)
    test_size = 0.2
    X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
        X_b, y_b, test_size=test_size, shuffle=False
    )
    print(f"学習データ数: {len(X_train_b)}, テストデータ数: {len(X_test_b)}")
    print(f"学習データ期間: {X_train_b.index.min()} ~ {X_train_b.index.max()}")
    print(f"テストデータ期間: {X_test_b.index.min()} ~ {X_test_b.index.max()}")

    # --- 3. データ準備 (スケーリング) ---
    print("\n--- 3. データ準備 (スケーリング) ---")
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_b)
    X_test_scaled = scaler.transform(X_test_b)
    print("スケーリング完了。")

    # --- 4. MLPモデルの定義 ---
    print("\n--- 4. MLPモデル定義 ---")
    n_features = X_train_scaled.shape[1]
    tf.random.set_seed(42) # TensorFlowの乱数シード固定

    model_mlp = keras.Sequential(
        [
            keras.Input(shape=(n_features,)),
            layers.BatchNormalization(),
            layers.Dense(128, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)), # L2正則化追加
            layers.Dropout(0.3),
            layers.BatchNormalization(),
            layers.Dense(64, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)), # L2正則化追加
            layers.Dropout(0.2),
            layers.BatchNormalization(),
            layers.Dense(32, activation="swish", kernel_regularizer=keras.regularizers.l2(0.001)), # L2正則化追加
            layers.Dropout(0.1),
            layers.Dense(1, activation="sigmoid"),
        ],
        name="mlp_set_b",
    )
    model_mlp.summary()

    # --- 5. モデルのコンパイル ---
    print("\n--- 5. モデルコンパイル ---")
    model_mlp.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss="binary_crossentropy",
                      metrics=[keras.metrics.AUC(name='auc'), 'accuracy'])

    # --- 6. モデルの学習 ---
    print("\n--- 6. モデル学習開始 ---")
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
    epochs = 100
    batch_size = 2048

    start_train_time = time.time()
    history = model_mlp.fit(X_train_scaled,
                            y_train_b,
                            batch_size=batch_size,
                            epochs=epochs,
                            validation_data=(X_test_scaled, y_test_b),
                            callbacks=[early_stopping],
                            verbose=1)
    end_train_time = time.time()
    print(f"学習完了。所要時間: {end_train_time - start_train_time:.2f} 秒")

    # --- 7. テストデータでの最終評価 ---
    print("\n--- 7. テストデータでの最終評価 ---")
    loss, auc_final_mlp, accuracy_final_mlp = model_mlp.evaluate(X_test_scaled, y_test_b, verbose=0)

    print(f"テストデータ Loss: {loss:.4f}")
    print(f"テストデータ Accuracy: {accuracy_final_mlp:.4f}")
    print(f"テストデータ AUC Score: {auc_final_mlp:.4f}") # ★これがMLPモデルの性能

    # --- 8. 予測確率の取得 (バックテスト用) ---
    print("\n--- 8. 予測確率の取得 ---")
    y_pred_proba_mlp = model_mlp.predict(X_test_scaled).flatten()
    print("予測確率取得完了。")

    # (オプション) Classification Report と Confusion Matrix
    y_pred_mlp = (y_pred_proba_mlp > 0.5).astype(int)
    print("\nClassification Report (閾値0.5):")
    print(classification_report(y_test_b, y_pred_mlp, target_names=['Low (0)', 'High (1)']))
    print("\nConfusion Matrix (閾値0.5):")
    conf_matrix_mlp = confusion_matrix(y_test_b, y_pred_mlp)
    conf_matrix_mlp_df = pd.DataFrame(conf_matrix_mlp, index=['Actual Low', 'Actual High'], columns=['Predicted Low', 'Predicted High'])
    display(conf_matrix_mlp_df)

    # 結果を保存
    if 'model_results' not in locals(): model_results = {}
    model_results['MLP_SetB_Final'] = {
        'model': model_mlp,
        'auc': auc_final_mlp,
        'accuracy': accuracy_final_mlp,
        'y_pred_proba': y_pred_proba_mlp,
        'features': 'Set B (ADR/PowerX)'
    }
    # モデルの保存 (任意)
    # model_mlp.save('mlp_model_setb_final.keras')

else:
    print(f"データフレーム df_processed_b_loaded の準備に失敗したか、'{csv_filename}' が見つかりませんでした。")

--- 1. CSVデータの読み込み ---
'BTCUSDT_5m_processed_set_b.csv' を読み込みました。
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 533505 entries, 2020-03-26 03:10:00+00:00 to 2025-04-21 13:50:00+00:00
Columns: 67 entries, open to MACDs_12_26_9
dtypes: float64(64), int64(3)
memory usage: 276.8 MB

--- 2. データ分割 ---
使用する特徴量の数: 59
学習データ数: 426804, テストデータ数: 106701
学習データ期間: 2020-03-26 03:10:00+00:00 ~ 2024-04-16 02:05:00+00:00
テストデータ期間: 2024-04-16 02:10:00+00:00 ~ 2025-04-21 13:50:00+00:00

--- 3. データ準備 (スケーリング) ---
スケーリング完了。

--- 4. MLPモデル定義 ---


Model: "mlp_set_b"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 59)             │           236 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         7,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,053 (74.43 KB)

 Trainable params: 18,551 (72.46 KB)

 Non-trainable params: 502 (1.96 KB)


--- 5. モデルコンパイル ---

--- 6. モデル学習開始 ---
Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5166 - auc: 0.5212 - loss: 0.8909 - val_accuracy: 0.5163 - val_auc: 0.5239 - val_loss: 0.7770
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5326 - auc: 0.5438 - loss: 0.7566 - val_accuracy: 0.5185 - val_auc: 0.5258 - val_loss: 0.7194
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5328 - auc: 0.5463 - loss: 0.7108 - val_accuracy: 0.5161 - val_auc: 0.5265 - val_loss: 0.7008
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5333 - auc: 0.5473 - loss: 0.6966 - val_accuracy: 0.5176 - val_auc: 0.5266 - val_loss: 0.6954
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5348 - auc: 0.5476 - loss: 0.6923 - val_accuracy: 0.5170 - val_auc: 0.5264 - val_loss: 0.6936
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5341 - auc: 0.5473 - loss: 0.6909 - val_accuracy: 0.5164 - val_auc: 0.5264 - val_loss: 

,Predicted Low,Predicted High
Actual Low,30726,22623
Actual High,28997,24355
